In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import itertools
import os
import pandas as pd
import sys
path_to_package = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(path_to_package)

In [13]:
df = pd.read_csv('../data/close.csv', index_col=0)
df = df.iloc[-50:, :3]

window = 4
corr = df.rolling(window).cov()

columns = ['{}/{}'.format(m, n) for (m, n) in itertools.product(df.columns, repeat=2)]

corr_flattened = pd.DataFrame(index=columns).transpose()

for i in range(df.shape[0]):

    data = corr[df.shape[1]*i : df.shape[1]*(i+1)].to_numpy().flatten()
    index = [corr.index[df.shape[1]*i][0]]

    temp = pd.DataFrame(data=data, columns=index, index=columns).transpose()
    corr_flattened = pd.concat([corr_flattened, temp])

corr_flattened.tail()

,MMM/MMM,MMM/ABT,MMM/ABBV,ABT/MMM,ABT/ABT,ABT/ABBV,ABBV/MMM,ABBV/ABT,ABBV/ABBV
2020-12-23,1.113287,0.587608,0.604265,0.587608,0.391294,0.272667,0.604265,0.272667,0.490197
2020-12-24,0.486291,0.143307,0.126876,0.143307,0.168092,-0.033241,0.126876,-0.033241,0.138157
2020-12-28,0.127666,0.107266,-0.033032,0.107266,0.181092,-0.062607,-0.033032,-0.062607,0.077490
2020-12-29,0.170292,0.003433,-0.174224,0.003433,0.192134,0.152435,-0.174224,0.152435,0.476490
2020-12-30,0.151692,-0.079643,-0.314301,-0.079643,0.087358,0.163868,-0.314301,0.163868,0.945797


In [16]:
def append_corr_matrix(df: pd.DataFrame,
                       window: int,
                       ) -> pd.DataFrame:
    """
        Computes the sliding correlation matrix of a multidimensional time series, \ 
        timewise flattens it and extracts just the upper triangular part (since it is symmetric), \
        then appends it to the initial time series.
    """

    columns = ['{}/{}'.format(m, n) for (m, n) in itertools.combinations_with_replacement(df.columns, r=2)]
    corr = df.rolling(window).cov()
    corr_flattened = pd.DataFrame(index=columns).transpose()

    for i in range(window-1, df.shape[0]):

        ind = np.triu_indices(df.shape[1])
        data = corr[df.shape[1]*i : df.shape[1]*(i+1)].to_numpy()[ind]
        index = [corr.index[df.shape[1]*i][0]]

        temp = pd.DataFrame(data=data, columns=index, index=columns).transpose()
        corr_flattened = pd.concat([corr_flattened, temp])

    return pd.concat([df.iloc[window-1 : ], corr_flattened], axis=1)

In [18]:
def append_corr_matrix2(df: pd.DataFrame,
                       window: int,
                       ) -> pd.DataFrame:
    """
        Computes the sliding correlation matrix of a multidimensional time series, \ 
        timewise flattens it and extracts just the upper triangular part (since it is symmetric), \
        then appends it to the initial time series.
    """

    columns = ['{}/{}'.format(m, n) for (m, n) in itertools.combinations_with_replacement(df.columns, r=2)]
    corr = df.rolling(window).cov()
    corr_flattened = pd.DataFrame(index=columns).transpose()

    for i in range(df.shape[0]):

        ind = np.triu_indices(df.shape[1])
        data = corr[df.shape[1]*i : df.shape[1]*(i+1)].to_numpy()[ind]
        index = [corr.index[df.shape[1]*i][0]]

        temp = pd.DataFrame(data=data, columns=index, index=columns).transpose()
        corr_flattened = pd.concat([corr_flattened, temp])

    return pd.concat([df, corr_flattened], axis=1).iloc[window-1 : ]

In [23]:
corr_flattened = append_corr_matrix(df, 4)
corr_flattened2 = append_corr_matrix2(df, 4)
print(corr_flattened.index)
print(corr_flattened2.index)

Index(['2020-10-23', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29',
       '2020-10-30', '2020-11-02', '2020-11-03', '2020-11-04', '2020-11-05',
       '2020-11-06', '2020-11-09', '2020-11-10', '2020-11-11', '2020-11-12',
       '2020-11-13', '2020-11-16', '2020-11-17', '2020-11-18', '2020-11-19',
       '2020-11-20', '2020-11-23', '2020-11-24', '2020-11-25', '2020-11-27',
       '2020-11-30', '2020-12-01', '2020-12-02', '2020-12-03', '2020-12-04',
       '2020-12-07', '2020-12-08', '2020-12-09', '2020-12-10', '2020-12-11',
       '2020-12-14', '2020-12-15', '2020-12-16', '2020-12-17', '2020-12-18',
       '2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24', '2020-12-28',
       '2020-12-29', '2020-12-30'],
      dtype='object')
Index(['2020-10-23', '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29',
       '2020-10-30', '2020-11-02', '2020-11-03', '2020-11-04', '2020-11-05',
       '2020-11-06', '2020-11-09', '2020-11-10', '2020-11-11', '2020-11-12',
       '2020-11-13

In [26]:
df = pd.read_csv('../data/close.csv', index_col=0)
df = df.iloc[-50:, :3]

window = 4

columns = ['Eigenvalue_{}'.format(m+1) for m in range(2)]
corr = df.rolling(window).cov()
corr_flattened = pd.DataFrame(index=columns).transpose()

for i in range(window-1, df.shape[0]):
    ind = np.triu_indices(df.shape[1])
    data = corr[df.shape[1]*i : df.shape[1]*(i+1)].to_numpy()
    data = np.linalg.eig(data)
    data = data[0]
    data[::-1].sort()
    data = data[:2]

    index = [corr.index[df.shape[1]*i][0]]
    temp = pd.DataFrame(data=data, columns=index, index=columns).transpose()
    corr_flattened = pd.concat([corr_flattened, temp])

pd.concat([df.iloc[window-1 : ], corr_flattened], axis=1)


,MMM,ABT,ABBV,Eigenvalue_1,Eigenvalue_2
2020-10-23,169.800003,107.790001,84.339996,3.148718,0.137980
2020-10-26,166.160004,109.000000,83.919998,3.940008,2.804801
2020-10-27,161.029999,108.580002,82.709999,19.868543,0.510592
2020-10-28,158.529999,106.419998,80.489998,28.572420,1.347709
2020-10-29,158.479996,105.000000,80.669998,18.216945,0.977576
2020-10-30,159.960007,105.110001,85.099998,5.519253,3.356264
2020-11-02,162.940002,107.480003,88.250000,18.670735,1.072825
2020-11-03,165.509995,109.559998,87.959999,24.832905,2.060276
2020-11-04,160.160004,111.529999,94.500000,22.364520,7.651177
2020-11-05,163.199997,113.529999,93.690002,19.547094,4.084112


In [ ]:
def append_corr_matrix_eigenvalues(df: pd.DataFrame,
                                   window: int,
                                   k: int = 10
                                   ) -> pd.DataFrame:
    
    columns = ['Eigenvalue_{}'.format(m+1) for m in range(k)]
    corr = df.rolling(window).cov()
    corr_eigenvalues = pd.DataFrame(index=columns).transpose()

    for i in range(window-1, df.shape[0]):
        data = corr[df.shape[1]*i : df.shape[1]*(i+1)].to_numpy()
        data = np.linalg.eig(data)
        data = data[0]
        data[::-1].sort()
        data = data[:k]

        index = [corr.index[df.shape[1]*i][0]]
        temp = pd.DataFrame(data=data, columns=index, index=columns).transpose()
        corr_eigenvalues = pd.concat([corr_eigenvalues, temp])

    return pd.concat([df.iloc[window-1 : ], corr_eigenvalues], axis=1)

In [2]:
df = pd.read_csv('../data/close.csv', index_col=0)

In [6]:
df.columns.shape

(470,)

In [25]:
with open("./tickers_S&P500.txt", "r") as file:
    lines = file.readlines()
    a = []
    for l in lines:
	    a.append(l.replace("\n", ""))

In [26]:
initial_number_of_shares = {key: 0 for key in a}

In [15]:
import json

In [27]:
with open("./initial_portfolio.json", "w") as file:
    file = json.dump(initial_number_of_shares, file)

['MMM',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'AYI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AET',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANDV',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'AJG',
 'AIZ',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BWA',
 'BXP',
 'BSX',
 'BHF',
 'BMY',
 'AVGO',
 'BB',
 'CHRW',
 'CA',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'CBOE',
 'KMX',
 'CCL',
 'CAT',
 'CNC',
 'CNP',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'COO',
 'GLW',
 'COST